### Importing the required libaries

In [93]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

### Load the cav to a pandas dataframe

In [94]:
df = pd.read_csv("universities_cleaned.csv", index_col=0)
df.head(5)

,Rank,Institution,Location,Continent,Founded,Affiliation,Delivery_Method,Enrollment,Link,location_city,location_country,updated_year
0,1,Indira Gandhi National Open University,"New Delhi, India",Asia,1985,Public,Distance/In-Person,7140000,https://en.wikipedia.org/wiki/Indira_Gandhi_Na...,New Delhi,India,2025
1,2,"National University, Bangladesh","Gazipur, Bangladesh",Asia,1992,Public,In-Person,3425832,https://en.wikipedia.org/wiki/National_Univers...,Gazipur,Bangladesh,2025
2,3,Anadolu University,"Eskişehir, Turkey",Asia,1958,Public,Distance/In-Person,1974343,https://en.wikipedia.org/wiki/Anadolu_University,Eskişehir,Turkey,2025
3,4,California Community Colleges,"California, United States",North America,1967,Public,In-Person,1800000,https://en.wikipedia.org/wiki/California_Commu...,California,United States,2025
4,5,Islamic Azad University,Iran,Asia,1982,Private,In-Person,1778000,https://en.wikipedia.org/wiki/Islamic_Azad_Uni...,NaN,Iran,2025


### Connect to Postgres

In [95]:
conn = psycopg2.connect(
    host='127.0.0.1',
    port=5433,
    password='P@ssw0rd.1',
    user='admin',
    database='amdari_db'
)

In [102]:
create_qry_file = open("sql_scripts/postgres_create_table.sql")
create_gry = create_qry_file.read()
create_gry

'CREATE TABLE universities(\n    id SERIAL PRIMARY KEY,\n    university_rank INT NOT NULL,\n    university_name VARCHAR(255) NOT NULL,\n    delivery_method VARCHAR(255) NOT NULL,\n    enrollment BIGINT NOT NULL,\n    founded INT NOT NULL,\n    affiliation VARCHAR(50) NOT NULL,\n    location_city VARCHAR(50) NOT NULL,\n    location_country VARCHAR(50) NOT NULL,\n    continent VARCHAR(50) NOT NULL,\n    uni_location VARCHAR(255) NOT NULL,\n    uni_link VARCHAR(300) NOT NULL,\n    updated_year INT NOT NULL,\n\n\n    UNIQUE (location_country, university_name, updated_year)\n);\n\nCREATE TABLE dimUniversities(\n    university_id SERIAL PRIMARY KEY,\n    university_name VARCHAR(255) NOT NULL,\n    delivery_method VARCHAR(255) NOT NULL,\n    founded INT NOT NULL,\n    affiliation VARCHAR(50) NOT NULL,\n    university_link VARCHAR(300) NOT NULL,\n\n    UNIQUE (university_name)\n);\n\nCREATE TABLE dimLocation(\n    location_id SERIAL PRIMARY KEY,\n    location_city VARCHAR(50) NOT NULL,\n    lo

In [ ]:
try:
    cursor = conn.cursor()
    cursor.execute(create_gry)
    conn.commit()
    cursor.close()
    print("Tables created sucessufully !!!")
except Exception as e:
    conn.rollback()  # Roll back on error
    cursor.close()
    print("Error executing query:", e)



In [104]:
engine = create_engine("postgresql+psycopg2://", creator=lambda:conn)

In [105]:
pd.read_sql("SELECT * from universities", con=engine)

,id,university_rank,university_name,delivery_method,enrollment,founded,affiliation,location_city,location_country,continent,uni_location,uni_link,updated_year


In [106]:
# Convert the data to tuples
data = df.itertuples(index=None, name=None)
# tuple(data)

### Load the data into the database

In [107]:
merge_qry_file = open("sql_scripts/postgres_upsert.sql")
merge_qry = merge_qry_file.read()
merge_qry


'INSERT INTO universities (\n  university_rank, university_name, uni_location, continent, founded,\n  affiliation, delivery_method, enrollment, uni_link,\n  location_city, location_country, updated_year\n)\nVALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)\nON CONFLICT (university_name, location_country, updated_year)\nDO UPDATE\nSET \n  university_rank = EXCLUDED.university_rank,\n  uni_location = EXCLUDED.uni_location,\n  continent = EXCLUDED.continent,\n  founded = EXCLUDED.founded,\n  affiliation = EXCLUDED.affiliation,\n  delivery_method = EXCLUDED.delivery_method,\n  enrollment = EXCLUDED.enrollment,\n  uni_link = EXCLUDED.uni_link,\n  location_city = EXCLUDED.location_city;\n\n'

In [ ]:
try:
    cursor = conn.cursor()
    cursor.executemany(merge_qry, data)
    conn.commit()
    cursor.close()
    print("Data loaded sucessufully !!!")
except Exception as e:
    conn.rollback()
    cursor.close()
    print("Error executing query:", e)

In [109]:
pd.read_sql("SELECT * from universities LIMIT 10", con=engine)

,id,university_rank,university_name,delivery_method,enrollment,founded,affiliation,location_city,location_country,continent,uni_location,uni_link,updated_year
0,1,1,Indira Gandhi National Open University,Distance/In-Person,7140000,1985,Public,New Delhi,India,Asia,"New Delhi, India",https://en.wikipedia.org/wiki/Indira_Gandhi_Na...,2025
1,2,2,"National University, Bangladesh",In-Person,3425832,1992,Public,Gazipur,Bangladesh,Asia,"Gazipur, Bangladesh",https://en.wikipedia.org/wiki/National_Univers...,2025
2,3,3,Anadolu University,Distance/In-Person,1974343,1958,Public,Eskişehir,Turkey,Asia,"Eskişehir, Turkey",https://en.wikipedia.org/wiki/Anadolu_University,2025
3,4,4,California Community Colleges,In-Person,1800000,1967,Public,California,United States,North America,"California, United States",https://en.wikipedia.org/wiki/California_Commu...,2025
4,5,5,Islamic Azad University,In-Person,1778000,1982,Private,NaN,Iran,Asia,Iran,https://en.wikipedia.org/wiki/Islamic_Azad_Uni...,2025
5,6,6,Allama Iqbal Open University,Distance/In-Person,1027000,1974,Public,Islamabad,Pakistan,Asia,"Islamabad, Pakistan",https://en.wikipedia.org/wiki/Allama_Iqbal_Ope...,2025
6,7,7,"Laureate Education, Inc.",Distance/In-Person,875000,1999,Private,NaN,International,Global,International,https://en.wikipedia.org/wiki/Laureate_Educati...,2025
7,8,8,Bangladesh Open University,Distance,650000,1992,Public,Gazipur,Bangladesh,Asia,"Gazipur, Bangladesh",https://en.wikipedia.org/wiki/Bangladesh_Open_...,2025
8,9,9,Universitas Terbuka,Distance,646467,1984,Public,Jakarta,Indonesia,Asia,"Jakarta, Indonesia",https://en.wikipedia.org/wiki/Universitas_Terbuka,2025
9,10,10,National Technological Institute of Mexico,In-Person,6200002019,1948,Public,NaN,Mexico,North America,Mexico,https://en.wikipedia.org/wiki/National_Technol...,2025


### Loading Dim University

In [114]:
df_universities = pd.read_sql("SELECT DISTINCT(u.university_name) as university_name,u.delivery_method,u.founded,u.affiliation,u.uni_link from universities u", con=engine)

df_universities.head(5)

,university_name,delivery_method,founded,affiliation,uni_link
0,Sapienza University of Rome,In-Person,1303,Public,https://en.wikipedia.org/wiki/Sapienza_Univers...
1,"National University, Bangladesh",In-Person,1992,Public,https://en.wikipedia.org/wiki/National_Univers...
2,Norte do Paraná University,Distance/In-Person,1972,Private,https://en.wikipedia.org/wiki/Universidade_Nor...
3,University of London,In-Person,1836,Public,https://en.wikipedia.org/wiki/University_of_Lo...
4,Rajiv Gandhi Technical University,In-Person,1998,Public,https://en.wikipedia.org/wiki/Rajiv_Gandhi_Tec...


In [124]:
df_universities.count()

university_name    90
delivery_method    90
founded            90
affiliation        90
uni_link           90
dtype: int64

In [133]:
#converts the datafram to a tuple
uni_data = df_universities.itertuples(index=None, name=None)
# tuple(uni_data)

In [ ]:
merge_dimUni_file = open("sql_scripts/upsert_dim_universities.sql")
merge_dimUni = merge_dimUni_file.read()
merge_dimUni

try:
    cursor = conn.cursor()
    cursor.executemany(merge_dimUni, uni_data)
    conn.commit()
    cursor.close()
    print("Data loaded sucessufully !!!")
except Exception as e:
    conn.rollback()
    cursor.close()
    print("Error executing query:", e)

### Loading DimLoaction

In [135]:
df_locations = pd.read_sql("select distinct u.location_city, u.location_country, u.continent from universities u ", con=engine)

df_locations.head(5)

,location_city,location_country,continent
0,"Bhopal, Madhya Pradesh",India,Asia
1,Ohio,United States,North America
2,Córdoba,Argentina,South America
3,NaN,Canada,North America
4,Madrid,Spain,Europe


In [139]:
df_locations.count()

location_city       74
location_country    74
continent           74
dtype: int64

In [149]:
#converts the datafram to a tuple
location_data = df_locations.itertuples(index=None, name=None)
# tuple(location_data)

In [150]:
merge_dimloc_file = open("sql_scripts/upsert_dim_locations.sql")
merge_dimloc = merge_dimloc_file.read()
merge_dimloc

try:
    cursor = conn.cursor()
    cursor.executemany(merge_dimloc, location_data)
    conn.commit()
    cursor.close()
    print("Data loaded sucessufully !!!")
except Exception as e:
    conn.rollback()
    cursor.close()
    print("Error executing query:", e)

Data loaded sucessufully !!!


### Loading the Fact Table

In [151]:
qry = """
    select r.university_rank, r.enrollment, du.university_id, l.location_id, r.updated_year
    from universities r
	    left join dimuniversities du on r. university_name = du.university_name 
	    left join dimlocation l on (r.location_city , r.location_country, r.continent) = (l.location_city,l.location_country, l.continent)
"""

df_fact_ranking = pd.read_sql(qry, con=engine)

df_fact_ranking.head(5)

,university_rank,enrollment,university_id,location_id,updated_year
0,1,7140000,15,24,2025
1,2,3425832,2,26,2025
2,3,1974343,51,14,2025
3,4,1800000,8,16,2025
4,5,1778000,36,43,2025


In [152]:
df_fact_ranking.count()

university_rank    90
enrollment         90
university_id      90
location_id        90
updated_year       90
dtype: int64

In [154]:
fact_data = df_fact_ranking.itertuples(index=None, name=None)
# tuple(fact_data)

In [155]:
insert_qry = """ 
        INSERT INTO factrankings 
            (university_rank,enrollment,university_id,location_id,updated_year)
        VALUES (%s, %s, %s, %s, %s)
    """

try:
    cursor = conn.cursor()
    cursor.executemany(insert_qry, fact_data)
    conn.commit()
    cursor.close()
    print("Data loaded sucessufully !!!")
except Exception as e:
    conn.rollback()
    cursor.close()
    print("Error executing query:", e)

Data loaded sucessufully !!!
